In [1]:
import pandas as pd 
from PIL import Image 
import cv2
import numpy as np
import os
import colorsys
import tqdm

# Get main colors

In [131]:
MAIN_COLORS_HSL = {
    'pink': (320, 340),
    'red': ((340, 360), (0, 15)), 'orange': (15, 45),
    'yellow': (45, 55), 'green': (55, 165),
    'blue': (165, 240),
    'violet': (240, 320)
    }

def find_neighbour_color(color_hsv, bl, wh):
    ans = ''
    if bl <= 15 and wh <= 15:
        return 'dark'
    for i, j in MAIN_COLORS_HSL.items():
        if i == 'red':
            if j[0][0] <= color_hsv <= j[0][1] or j[1][0] <= color_hsv <= j[1][1]:
                ans = 'red'
                break
        elif j[0] <= color_hsv <= j[1]:
            ans = i
            break
    if ans == 'red' and ((bl <= 28 and wh >= 78) or wh >= 90):
        ans = 'pink'
    elif ans == 'orange' and  (bl <= 28 and wh >= 78 or wh >= 90):
        ans = 'peach'
    elif (ans == 'orange' or ans == 'red') and (wh <= 25 or (wh <= 45 and bl <= 45)):
        ans = 'brown'
    return ans

In [ ]:
data = pd.read_csv('data.csv', delimiter=';')
data['bl'] = data['bl'] * 100
data['wh'] = data['wh'] / 255 * 100
data['color'] = data.apply(lambda x: find_neighbour_color(x.color_hsv, x.bl, x.wh), axis=1)
data

In [147]:
data.groupby('color').img_path.count()

color
blue      1247
brown      798
dark       492
green      737
orange     986
peach     1314
pink       797
red        575
violet     412
yellow     188
Name: img_path, dtype: int64

In [153]:
for i in data[data.color == 'peach'].img_path[200:400]:
    im = Image.open(i) 
    im.show()


In [155]:
data.to_csv('data_with_colors.csv', index=False)

# Clothes type

In [9]:
import os

In [ ]:
python3 train.py --epochs 100 --workers 4 --batch-size 32 --data data/clothes.yaml --img 640 640 --cfg cfg/training/yolov7_clothes-tiny.yaml --weights 'yolov7-tiny.pt' --name yolov7_tiny_clothes_fixed_res --hyp data/hyp.scratch.tiny.yaml

In [56]:
data = pd.read_csv('data_with_colors.csv')
data.img_path = data.img_path.apply(lambda x: x.split('/')[1])
data

,img_path,color_hsv,bl,wh,color
0,image6561.png,40.000000,1.209677,97.254902,peach
1,image10104.png,28.085106,20.524017,89.803922,peach
2,image9652.png,215.555556,27.551020,38.431373,blue
3,image3419.png,290.000000,3.260870,72.156863,violet
4,image5068.png,172.500000,19.047619,16.470588,blue
...,...,...,...,...,...
7541,image3400.png,141.818182,23.913043,18.039216,green
7542,image8555.png,16.000000,64.516129,36.470588,orange
7543,image9893.png,169.565217,19.327731,46.666667,blue
7544,image1217.png,354.000000,89.285714,21.960784,brown


In [57]:
classes = list(map(str.strip, open('classes.txt').readlines()))
classes

['аксесуары',
 'ботинки',
 'верхняя одежда',
 'джинсы',
 'жилетка',
 'комбинезон',
 'кофта',
 'кроссовки',
 'майка',
 'пиджак',
 'платье',
 'рубашка',
 'рюкзак',
 'сумка',
 'толстовка',
 'туфли',
 'футболка',
 'шляпа',
 'шорты',
 'штаны',
 'юбка']

In [58]:
images = next(os.walk('images'))[2]
data = data[data.img_path.isin(list(map(lambda x: x.split('-')[-1], images)))]

In [59]:
labels = next(os.walk('labels'))[2]
a = []
for i in labels:
    clth = list(map(int, open('labels/'+i).read().replace('\n', ' ').split()[::5]))
    a.append((i.replace('txt', 'png'), clth))
labels_data = pd.DataFrame(a)
labels_data['img_path'] = labels_data[0].apply(lambda x: x.split('-')[-1])


In [63]:
data = labels_data.set_index('img_path').join(
    data.set_index('img_path')).reset_index().drop(
        columns='img_path').rename(columns={0: 'img_path', 1: 'clothes'})
data

,img_path,clothes,color_hsv,bl,wh,color
0,image10006.png,"[6, 20, 1, 13, 0]",60.000000,0.408163,96.078431,green
1,image10589.png,"[3, 1, 6]",27.272727,4.526749,95.294118,peach
2,8c731166-image28.png,"[20, 15, 6]",25.090909,56.701031,38.039216,orange
3,66084130-image8.png,"[10, 15, 0, 13]",45.000000,1.581028,99.215686,peach
4,8471d3a7-image82.png,"[10, 2, 13, 1, 1]",20.000000,29.268293,16.078431,brown
...,...,...,...,...,...,...
526,image10382.png,"[18, 13, 8, 0, 0, 7, 0]",240.000000,4.347826,9.019608,dark
527,image10221.png,"[10, 1, 0, 0]",38.461538,21.195652,72.156863,orange
528,image10584.png,"[5, 0, 0, 0, 7, 6]",37.402597,38.888889,77.647059,orange
529,image10023.png,"[10, 15, 13]",204.878049,19.902913,80.784314,blue
